<a href="https://colab.research.google.com/github/abobakrbayomi-cell/Medical-Data-Analysis/blob/main/Medical_Data_Analysis_Project_(Git_Hup)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
from google.colab import files
import os


In [ ]:
print("Please upload the CSV file (Data.xlsx - Clean Data.csv)")
uploaded = files.upload()

In [ ]:
filename = list(uploaded.keys())[0]

if filename.endswith('.xlsx') or filename.endswith('.xls'):
    df = pd.read_excel(io.BytesIO(uploaded[filename]), sheet_name='Clean Data')
    print("\n تم تحميل شيت (Clean Data) بنجاح! .. ودي عينة منه:")
    display(df.head())
else:
    print("الملف اللي رفعته مش إكسيل! حاول ترفع ملف الامتداد بتاعه .xlsx")

# **Discriptive Statistics**

In [ ]:


import pandas as pd
import numpy as np

# 1. تحميل الداتا
try:
    # يفضل استخدام اسم الملف الموجود عندك
    df = pd.read_excel('Data.xlsx', sheet_name='Clean Data')

    # تنظيف عمود Group
    df['Group'] = df['Group'].astype(str).str.replace('"', '').str.strip()

    # --- التعديل: خريطة تغيير الأسماء ---
    group_map = {
        'Group 1': 'Control',
        'Group 2': 'Newly Diagnosed Patients',
        'Group 3': 'Treated Patients'
    }
    df['Group'] = df['Group'].replace(group_map)
    print("تم تحديث أسماء المجموعات.")

except Exception as e:
    print(f"⚠️ تأكد من تحميل ملف البيانات الصحيح! الخطأ: {e}")

# تحديد الأعمدة الرقمية (يفضل تحديدها بالاسم لضمان الدقة)
numeric_cols = ['HO-1', 'CP', 'CEA', 'Hb', 'Creatinine', 'ALT', 'Age', 'Cu', 'Fe', 'Zn', 'Ca']

# التأكد من تحويلها لأرقام
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 2. دالة حساب الإحصائيات (شاملة Q1, Q3, IQR)
def calculate_detailed_stats(series):
    if series.empty: return {}

    mean = series.mean()
    sem = series.sem()
    sd = series.std()
    minimum = series.min()
    q1 = series.quantile(0.25)
    median = series.median()
    q3 = series.quantile(0.75)
    maximum = series.max()
    rng = maximum - minimum
    iqr = q3 - q1

    return {
        'Mean ± SE': f"{mean:.2f} ± {sem:.2f}",
        'SD': round(sd, 2),
        'Minimum': round(minimum, 2),
        'Q1': round(q1, 2),
        'Median': round(median, 2),
        'Q3': round(q3, 2),
        'Maximum': round(maximum, 2),
        'Range': round(rng, 2),
        'IQR': round(iqr, 2)
    }

# ---------------------------------------------------------
# الجدول الأول: Total (كل الداتا)
# ---------------------------------------------------------
total_rows = []
for col in numeric_cols:
    stats = calculate_detailed_stats(df[col].dropna())
    row = {'Variable': col}
    row.update(stats)
    total_rows.append(row)

df_total_final = pd.DataFrame(total_rows)

# ترتيب الأعمدة
cols_order_total = ['Variable', 'Mean ± SE', 'SD', 'Minimum', 'Q1', 'Median', 'Q3', 'Maximum', 'Range', 'IQR']
df_total_final = df_total_final[cols_order_total]


# ---------------------------------------------------------
# الجدول الثاني: Groups (كل متغير تحته الجروبات)
# ---------------------------------------------------------
groups_rows = []
# تحديد الترتيب المرغوب للعرض
desired_order = ['Control', 'Newly Diagnosed Patients', 'Treated Patients']

for col in numeric_cols:
    for group in desired_order:
        # التأكد من وجود المجموعة في الداتا لتجنب الأخطاء
        if group in df['Group'].unique():
            sub_series = df[df['Group'] == group][col].dropna()
            stats = calculate_detailed_stats(sub_series)

            row = {'Variable': col, 'Group': group}
            row.update(stats)
            groups_rows.append(row)

df_groups_final = pd.DataFrame(groups_rows)

# ترتيب الأعمدة
cols_order_groups = ['Variable', 'Group', 'Mean ± SE', 'SD', 'Minimum', 'Q1', 'Median', 'Q3', 'Maximum', 'Range', 'IQR']
df_groups_final = df_groups_final[cols_order_groups]


# 3. الحفظ في ملف إكسيل
output_filename = 'Descriptive_Statistics_Renamed.xlsx'
with pd.ExcelWriter(output_filename) as writer:
    df_total_final.to_excel(writer, sheet_name='Total', index=False)
    df_groups_final.to_excel(writer, sheet_name='Groups Analysis', index=False)

print(f"✅ تم إنشاء الملف بنجاح: {output_filename}")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.patches import Patch
import numpy as np

# اسم الملف
df = pd.read_excel('Data.xlsx', sheet_name='Clean Data')

# The original 'try' block attempting to load a CSV has been removed
# to resolve the IndentationError and ensure consistent Excel file loading.

# 2. تنظيف أسماء المجموعات
df['Group'] = df['Group'].astype(str).str.replace('"', '').str.strip()

# 3. تغيير الأسماء حسب الطلب
group_map = {
    'Group 1': 'Control',
    'Group 2': 'Newly Diagnosed Patients',
    'Group 3': 'Treated Patients'
}
df['Group'] = df['Group'].replace(group_map)

# 4. تنظيف عمود الجنس
df['Sex'] = df['Sex'].astype(str).str.strip().str.upper()
sex_map = {'M': 'Male', 'F': 'Female', 'MALE': 'Male', 'FEMALE': 'Female'}
df['Sex'] = df['Sex'].replace(sex_map)

# حساب العدد الكلي لكل جنس (للـ Legend)
sex_totals = df['Sex'].value_counts()

# إنشاء نسخة 'Total' للعينة الكلية
df_total = df.copy()
df_total['Group'] = 'Total'
df_combined = pd.concat([df, df_total], ignore_index=True)

# --- تجهيز البيانات للرسم (حساب النسب) ---
# 1. حساب عدد كل جنس داخل كل مجموعة
df_counts = df_combined.groupby(['Group', 'Sex']).size().reset_index(name='Count')

# 2. حساب العدد الكلي لكل مجموعة (للقسمة عليه)
group_counts_lookup = df_counts.groupby('Group')['Count'].sum()

# 3. حساب النسبة المئوية
df_agg = df_counts.copy()
df_agg['Percentage'] = df_agg.apply(lambda x: x['Count'] / group_counts_lookup[x['Group']], axis=1)

# --- الرسم ---
sns.set(style='whitegrid')
plt.figure(figsize=(14, 8))

# ترتيب المحور الأفقي
group_order = ['Control', 'Newly Diagnosed Patients', 'Treated Patients', 'Total']

ax = sns.barplot(
    data=df_agg,
    x='Group',
    y='Percentage',
    hue='Sex',
    order=group_order,
    palette='Greys', # ألوان رمادية
    edgecolor='black'
)

# تنسيق المحور الصادي كنسبة مئوية
ax.set_ylabel('Percentage', fontsize=14, weight='bold')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: f'{y:.0%}'))
ax.set_ylim(0, 1.15) # زيادة المساحة العلوية للتسميات

# --- إضافة الأرقام والنسب فوق الأعمدة ---
hue_order = sorted(df_agg['Sex'].unique()) # ['Female', 'Male']

for i, container in enumerate(ax.containers):
    hue_label = hue_order[i]
    hue_data = df_agg[df_agg['Sex'] == hue_label]

    # إعادة ترتيب البيانات لتطابق الرسم
    hue_data = hue_data.set_index('Group').reindex(group_order)

    labels = []
    for group_name in group_order:
        if group_name in hue_data.index and pd.notna(hue_data.loc[group_name, 'Count']):
            count = int(hue_data.loc[group_name, 'Count'])
            pct = hue_data.loc[group_name, 'Percentage']
            labels.append(f"{count} ({pct:.1%})")
        else:
            labels.append("0")

    ax.bar_label(container, labels=labels, fontsize=10, color='black', padding=3, weight='bold')

# --- تحسين مفتاح الرسم (Legend) ---
handles, labels = ax.get_legend_handles_labels()
new_handles = []
new_labels = []

for j, label_text in enumerate(labels):
    total_count = sex_totals.get(label_text, 0)
    new_label_text = f"{label_text} (Total: {total_count})"
    new_labels.append(new_label_text)

    # الحفاظ على اللون الأصلي
    facecolor = handles[j].get_facecolor() if hasattr(handles[j], 'get_facecolor') else 'gray'

    new_handles.append(
        Patch(facecolor=facecolor, edgecolor='black', label=new_label_text)
    )

ax.legend(
    handles=new_handles,
    labels=new_labels,
    title='Sex (with Total Count)',
    loc='upper right',
    fontsize=11,
    title_fontsize=12
)

# العناوين النهائية
ax.set_xlabel('Study Group', fontsize=14, weight='bold')
plt.xticks(fontsize=11)
plt.tight_layout()

# حفظ الصورة
plt.savefig('sex_distribution_percentage_plot.png', dpi=300)
print("Plot saved as 'sex_distribution_percentage_plot.png'")

# **Normality (Shapiro-Wilk)**

In [ ]:
try:
    df = pd.read_excel('Data.xlsx', sheet_name='Clean Data')

    # تنظيف الأسماء لضمان الدقة
    df['Group'] = df['Group'].astype(str).str.strip().str.replace('"', '').str.replace("'", "")
    print("✅ تم تحميل البيانات بنجاح.")
except:
    print("❌ خطأ: مش لاقي ملف Data.xlsx. تأكد إنه مرفوع.")

# 2. إعداد الجدول للنتائج
results = []
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Stage' in numeric_cols: numeric_cols.remove('Stage')

groups = sorted(df['Group'].unique())

print("\n🧪 جاري فحص الاعتدالية (Normality Test) للمجموعات + التوتال...\n")

for col in numeric_cols:
    row = {'Variable': col}
    is_group_normal = True # عشان نحدد التوصية بناءً على الجروبات

    # --- أولاً: فحص كل جروب لوحده ---
    for g in groups:
        data = df[df['Group'] == g][col].dropna()

        if len(data) >= 3:
            try:
                stat, p_value = stats.shapiro(data)
                status = "Normal" if p_value > 0.05 else "Not Normal"

                # لو جروب واحد مش طبيعي، يبقى التوصية Non-Parametric
                if p_value <= 0.05:
                    is_group_normal = False

                row[f'{g} (P-value)'] = f"{p_value:.4f} ({status})"
            except:
                row[f'{g} (P-value)'] = "Error"
        else:
            row[f'{g} (P-value)'] = "N<3"

    # --- ثانياً: فحص التوتال (كل الداتا مع بعض) ---
    total_data = df[col].dropna()
    if len(total_data) >= 3:
        try:
            stat_t, p_val_t = stats.shapiro(total_data)
            status_t = "Normal" if p_val_t > 0.05 else "Not Normal"
            row['Total (P-value)'] = f"{p_val_t:.4f} ({status_t})"
        except:
            row['Total (P-value)'] = "Error"
    else:
        row['Total (P-value)'] = "N<3"

    # --- التوصية النهائية ---
    # (عادة بنعتمد على الجروبات عشان نختار بين ANOVA و Kruskal)
    if is_group_normal:
        row['Suggestion'] = 'Parametric (ANOVA)'
    else:
        row['Suggestion'] = 'Non-Parametric (Kruskal-Wallis)'

    results.append(row)

# 3. حفظ وعرض النتائج
normality_df = pd.DataFrame(results)

# دالة تلوين عشان الملف يبقى شكله حلو
def highlight_cells(val):
    val_str = str(val)
    if 'Not Normal' in val_str:
        return 'color: red; font-weight: bold'
    elif 'Normal' in val_str:
        return 'color: green'
    return ''

print("--- جدول نتائج اختبار Shapiro-Wilk (Groups & Total) ---")
display(normality_df.style.map(highlight_cells))

# تصدير للإكسيل
output_file = 'Shapiro_Normality_With_Total.xlsx'
normality_df.to_excel(output_file, index=False)
print(f"\n✅ تم حفظ الملف الجديد باسم: {output_file}")

# **Kruskal-Wallis**

In [ ]:
# 1. تثبيت المكتبة اللازمة لحساب الحروف (Post-hoc)
!pip install scikit-posthocs

import pandas as pd
import numpy as np
from scipy import stats
import scikit_posthocs as sp

# 2. تحميل البيانات
try:
    df = pd.read_excel('Data.xlsx', sheet_name='Clean Data')
    # تنظيف أسماء الجروبات
    df['Group'] = df['Group'].astype(str).str.strip().str.replace('"', '').str.replace("'", "")
    print("✅ تم تحميل البيانات بنجاح.")
except:
    print("⚠️ تأكد من رفع ملف Data.xlsx")

# 3. دالة تحديد الحروف (Significance Letters)
def get_letters(p_values, groups):
    # دالة ذكية لتوزيع الحروف a, b, c بناءً على الفروقات
    g1, g2, g3 = groups[0], groups[1], groups[2]

    try:
        p12 = p_values.loc[g1, g2]
        p13 = p_values.loc[g1, g3]
        p23 = p_values.loc[g2, g3]
    except:
        return {g: 'a' for g in groups}

    # مصفوفة الحروف المبدئية
    letters = {g1: '', g2: '', g3: ''}

    # لو مفيش أي فروق
    if p12 > 0.05 and p13 > 0.05 and p23 > 0.05:
        return {g: 'a' for g in groups}

    # لو كله مختلف عن بعضه
    if p12 <= 0.05 and p13 <= 0.05 and p23 <= 0.05:
        return {g1: 'a', g2: 'b', g3: 'c'}

    # حالات التداخل
    # 1 vs 2
    if p12 > 0.05: # زي بعض
        letters[g1] += 'a'
        letters[g2] += 'a'
    else: # مختلفين
        letters[g1] += 'a'
        letters[g2] += 'b'

    # 3 (المجموعة التالتة)
    # نشوف علاقتها بـ 1
    if p13 > 0.05: # زي 1
        if 'a' in letters[g1]: letters[g3] += 'a'
        else: letters[g3] += 'b' # (نادرة)

    # نشوف علاقتها بـ 2
    if p23 > 0.05: # زي 2
        if 'b' in letters[g2] and 'b' not in letters[g3]: letters[g3] += 'b'
        elif 'a' in letters[g2] and 'a' not in letters[g3]: letters[g3] += 'a'

    # لو 3 مختلفة عن الاتنين
    if p13 <= 0.05 and p23 <= 0.05:
        letters[g3] = 'c'

    # تنظيف الحروف (ترتيب أبجدي)
    for g in letters:
        letters[g] = "".join(sorted(set(letters[g])))
        if letters[g] == "": letters[g] = "c" # fallback

    return letters

# 4. التجهيز والحساب
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Stage' in numeric_cols: numeric_cols.remove('Stage')

groups_order = sorted(df['Group'].unique())
results = []

print("\n🚀 جاري حساب Kruskal-Wallis & Post-hoc Letters...\n")

for col in numeric_cols:
    # تجميع الداتا
    g1_data = df[df['Group'] == groups_order[0]][col].dropna()
    g2_data = df[df['Group'] == groups_order[1]][col].dropna()
    g3_data = df[df['Group'] == groups_order[2]][col].dropna()
    all_data = [g1_data, g2_data, g3_data]

    # أ) Levene's Test (Homogeneity)
    try:
        stat_lev, p_lev = stats.levene(*all_data)
        lev_res = f"{p_lev:.3f}" + (" *" if p_lev < 0.05 else "")
    except:
        lev_res = "NaN"

    # ب) Kruskal-Wallis Test
    try:
        stat_k, p_k = stats.kruskal(*all_data)
        k_res_stat = round(stat_k, 3)
        k_res_p = f"{p_k:.3f}" + (" *" if p_k < 0.05 else "")
    except:
        k_res_stat, k_res_p = "NaN", "NaN"

    # ج) Post-hoc Letters (Dunn's Test)
    letter_map = {g: 'a' for g in groups_order}
    if p_k < 0.05: # لو فيه فرق معنوي نحسب الحروف
        try:
            ph_data = df[['Group', col]].dropna()
            p_matrix = sp.posthoc_dunn(ph_data, val_col=col, group_col='Group', p_adjust='bonferroni')
            letter_map = get_letters(p_matrix, groups_order)
        except:
            pass # Keep as 'a' if error

    # د) تجهيز الصف في الجدول
    row = {'Variable': col}

    for i, g_name in enumerate(groups_order):
        data = all_data[i]
        if data.empty:
            row[f'Median ± SIQR ("{g_name}")'] = "-"
            continue

        median = data.median()
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        siqr = (q3 - q1) / 2 # معادلة SIQR

        ltr = letter_map[g_name]
        # الصيغة النهائية: Median ± SIQR (Letter)
        row[f'Median ± SIQR ("{g_name}")'] = f"{median:.2f} ± {siqr:.2f} ({ltr})"

    # باقي الأعمدة
    row["Normality (Levene's test)"] = lev_res
    row['Kruskal_H_Statistic'] = k_res_stat
    row['Kruskal_P_Value'] = k_res_p

    results.append(row)

# 5. الحفظ
final_df = pd.DataFrame(results)
filename = 'Kruskal_Analysis_Final.xlsx'
final_df.to_excel(filename, index=False)

print(f"🎉 تم الانتهاء! الملف جاهز: {filename}")
display(final_df.head())

# **ANOVA Test**

In [ ]:


import pandas as pd
import numpy as np
from scipy import stats
import scikit_posthocs as sp
import os

# 2. تحميل البيانات
filename = 'Data.xlsx'
try:
    if os.path.exists(filename):
        df = pd.read_excel(filename, sheet_name='Clean Data')
    else:
        csv_files = [f for f in os.listdir('.') if 'clean' in f.lower() and f.endswith('.csv')]
        if csv_files:
            df = pd.read_csv(csv_files[0])
        else:
            raise FileNotFoundError("الملف غير موجود")

    df['Group'] = df['Group'].astype(str).str.strip().str.replace('"', '').str.replace("'", "")
    print("✅ تم تحميل البيانات.")
except Exception as e:
    print(f"❌ خطأ: {e}")
    df = pd.DataFrame()

# 3. دالة تنسيق P-value
def format_p(p):
    if pd.isna(p): return "-"
    if p < 0.001: return "< 0.001 *"
    sig = " *" if p < 0.05 else ""
    return f"{p:.3f}{sig}"

# 4. دالة الحروف (نفس اللوجيك)
def get_letters(p_values, groups):
    g1, g2, g3 = groups[0], groups[1], groups[2]
    letters = {g1: '', g2: '', g3: ''}
    try:
        p12, p13, p23 = p_values.loc[g1, g2], p_values.loc[g1, g3], p_values.loc[g2, g3]
    except: return {g: 'a' for g in groups}

    if p12 > 0.05 and p13 > 0.05 and p23 > 0.05: return {g: 'a' for g in groups}
    if p12 <= 0.05 and p13 <= 0.05 and p23 <= 0.05: return {g1: 'a', g2: 'b', g3: 'c'}

    # تداخلات
    if p12 > 0.05: letters[g1]+='a'; letters[g2]+='a'
    else: letters[g1]+='a'; letters[g2]+='b'

    match_1, match_2 = (p13 > 0.05), (p23 > 0.05)
    if match_1: letters[g3] += ('a' if 'a' in letters[g1] else 'b')
    if match_2:
        if 'b' in letters[g2] and 'b' not in letters[g3]: letters[g3] += 'b'
        elif 'a' in letters[g2] and 'a' not in letters[g3]: letters[g3] += 'a'

    if not match_1 and not match_2: letters[g3] = 'c'
    for g in letters:
        cleaned = "".join(sorted(set(letters[g])))
        letters[g] = cleaned if cleaned else "c"
    return letters

# 5. التنفيذ (ANOVA + Tukey) بدون أقواس
if not df.empty:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'Stage' in numeric_cols: numeric_cols.remove('Stage')
    groups_order = sorted(df['Group'].unique())
    results = []

    print("\n🚀 جاري حساب ANOVA + Letters (بدون أقواس)...\n")

    for col in numeric_cols:
        g1_data = df[df['Group'] == groups_order[0]][col].dropna()
        g2_data = df[df['Group'] == groups_order[1]][col].dropna()
        g3_data = df[df['Group'] == groups_order[2]][col].dropna()
        all_data = [g1_data, g2_data, g3_data]

        # Levene
        try:
            stat_lev, p_lev = stats.levene(*all_data)
            lev_res = format_p(p_lev)
        except: lev_res = "-"

        # ANOVA
        try:
            f_stat, p_anova = stats.f_oneway(*all_data)
            f_res = f"{f_stat:.2f}"
            p_res = format_p(p_anova)
        except: f_res, p_res, p_anova = "-", "-", 1.0

        # Tukey Letters
        letter_map = {g: 'a' for g in groups_order}
        if p_anova < 0.05:
            try:
                ph_data = df[['Group', col]].dropna()
                p_matrix = sp.posthoc_tukey(ph_data, val_col=col, group_col='Group')
                letter_map = get_letters(p_matrix, groups_order)
            except: pass

        # تعبئة الجدول
        row = {'Variable': col}
        for i, g_name in enumerate(groups_order):
            data = all_data[i]
            if data.empty: row[f'Mean ± SD ("{g_name}")'] = "-"; continue

            mean_val = data.mean()
            sd_val = data.std()

            # التعديل هنا: شيلنا الأقواس ()
            # النتيجة هتبقى: 12.5 ± 2.1 a
            row[f'Mean ± SD ("{g_name}")'] = f"{mean_val:.2f} ± {sd_val:.2f} {letter_map[g_name]}"

        row["Normality (Levene's test)"] = lev_res
        row['F-ratio'] = f_res
        row['P-value'] = p_res
        results.append(row)

    # الحفظ
    anova_df = pd.DataFrame(results)
    output_file = 'ANOVA_Results_No_Parentheses.xlsx'
    anova_df.to_excel(output_file, index=False)

    print(f"🎉 تم الحفظ بنجاح: {output_file}")
    display(anova_df.head())

# **Bar Chart**

# **Bar chart with post hoc, Mean and SD**

In [ ]:


if 'anova_df' not in locals():
    print("⚠️ جدول anova_df مش موجود!")
else:
    print(f"🚀 جاري رسم {len(anova_df)} بار شارت بالتصميم الجديد (Legend فوق يمين)...")

    groups_labels_on_chart = ['Control', 'NDP', 'TP']
    group_cols = [c for c in anova_df.columns if 'Mean ± SD' in c]

    # إعدادات الألوان والأنماط
    colors = ['#D3D3D3', '#A9A9A9', '#808080']
    patterns = ['/', '\\', 'x']

    # تجهيز عناصر الـ Legend مرة واحدة عشان نستخدمها في كل رسمة
    # ده بيخلي البوكس يظهر فيه شكل المربع الصغير وجنبه الكلام
    legend_handles = [
        mpatches.Patch(facecolor=colors[0], hatch=patterns[0], edgecolor='black', label='Control'),
        mpatches.Patch(facecolor=colors[1], hatch=patterns[1], edgecolor='black', label='NDP: Newly Diagnosed Patients'),
        mpatches.Patch(facecolor=colors[2], hatch=patterns[2], edgecolor='black', label='TP: Treated Patients')
    ]

    for index, row in anova_df.iterrows():
        var_name = row['Variable']
        print(f"  - Drawing: {var_name}...")

        means = []
        sds = []
        letters = []

        for col in group_cols:
            val_str = str(row[col])
            try:
                parts = val_str.split()
                means.append(float(parts[0]))
                sds.append(float(parts[2]))
                letters.append(parts[3])
            except:
                means.append(0)
                sds.append(0)
                letters.append('')

        df_plot = pd.DataFrame({
            'Group': groups_labels_on_chart,
            'Mean': means,
            'SD': sds,
            'Letter': letters
        })

        # --- الرسم ---
        fig, ax = plt.subplots(figsize=(10, 7)) # رجعنا الحجم الطبيعي لأن الـ Legend بقى جوة

        bars = ax.bar(
            df_plot['Group'],
            df_plot['Mean'],
            yerr=df_plot['SD'],
            capsize=5,
            color=colors,
            edgecolor='black',
            linewidth=1.0,
            alpha=1.0
        )

        for bar, pattern in zip(bars, patterns):
            bar.set_hatch(pattern)

        ax.set_xlabel('Group', fontsize=12, fontweight='bold')
        ax.set_ylabel(f'Mean {var_name}', fontsize=12, fontweight='bold')

        # تحسين شكل الشبكة
        ax.grid(axis='y', linestyle=':', alpha=0.5, color='gray')
        ax.set_axisbelow(True)

        max_y = (df_plot['Mean'] + df_plot['SD']).max()
        if pd.isna(max_y) or max_y == 0: max_y = 1
        ax.set_ylim(0, max_y * 1.35)

        # إضافة النصوص (Mean + SD + Letters)
        for i, bar in enumerate(bars):
            r = df_plot.iloc[i]
            height = bar.get_height()
            if pd.isna(height): height = 0

            label_text = f"({r['Letter']})\nMean: {r['Mean']}\nSD: {r['SD']}"

            ax.text(
                bar.get_x() + bar.get_width() / 2,
                height + r['SD'] + (max_y * 0.02),
                label_text,
                ha='center',
                va='bottom',
                fontsize=10,
                fontweight='bold',
                color='black'
            )

        # --- إضافة الـ Legend (البوكس التعريفي) ---
        # loc='upper right' : مكان البوكس
        # prop={'size': 9} : حجم الخط أصغر
        # framealpha=0.9 : شفافية خلفية البوكس
        ax.legend(
            handles=legend_handles,
            loc='upper right',
            fontsize=10,
            frameon=True,
            edgecolor='black',
            framealpha=1,
            fancybox=False # يخلي الحواف مربعة مش مدورة
        )

        plt.tight_layout()

        safe_var_name = "".join([c for c in var_name if c.isalnum() or c in (' ', '-', '_')]).strip()
        plt.savefig(f'BarChart_{safe_var_name}.png', dpi=300)
        plt.close()

    print("\n✅ تم حفظ جميع الرسومات (Legend احترافي)!")

# **Custom Post Hoc**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np

var_name = "CP"

if var_name not in anova_df['Variable'].values:
    print(f"⚠️ المتغير {var_name} مش موجود!")
else:
    target_row = anova_df[anova_df['Variable'] == var_name].iloc[0]
    print(f"📊 جاري رسم {var_name}...")

    groups_labels_on_chart = ['Control', 'NDP', 'TP']
    group_cols = [c for c in anova_df.columns if 'Mean ± SD' in c]

    means = []
    sds = []
    for col in group_cols:
        val_str = str(target_row[col])
        try:
            parts = val_str.split()
            means.append(float(parts[0]))
            sds.append(float(parts[2]))
        except:
            means.append(0)
            sds.append(0)

    # الحروف المخصصة
    custom_letters = ['ab', 'a', 'b']

    df_plot = pd.DataFrame({
        'Group': groups_labels_on_chart,
        'Mean': means,
        'SD': sds,
        'Letter': custom_letters
    })

    # --- الرسم ---
    fig, ax = plt.subplots(figsize=(10, 7))

    colors = ['#D3D3D3', '#A9A9A9', '#808080']
    patterns = ['/', '\\', 'x']

    bars = ax.bar(
        df_plot['Group'],
        df_plot['Mean'],
        yerr=df_plot['SD'],
        capsize=5,
        color=colors,
        edgecolor='black',
        linewidth=1.0,
        alpha=1.0
    )

    for bar, pattern in zip(bars, patterns):
        bar.set_hatch(pattern)

    ax.set_xlabel('Group', fontsize=12, fontweight='bold')
    ax.set_ylabel(f'Mean {var_name}', fontsize=12, fontweight='bold')
    ax.grid(axis='y', linestyle=':', alpha=0.5, color='gray')
    ax.set_axisbelow(True)

    max_y = (df_plot['Mean'] + df_plot['SD']).max()
    if pd.isna(max_y) or max_y == 0: max_y = 1
    ax.set_ylim(0, max_y * 1.35)

    for i, bar in enumerate(bars):
        row = df_plot.iloc[i]
        height = bar.get_height()
        if pd.isna(height): height = 0

        label_text = f"({row['Letter']})\nMean: {row['Mean']}\nSD: {row['SD']}"

        ax.text(
            bar.get_x() + bar.get_width() / 2,
            height + row['SD'] + (max_y * 0.02),
            label_text,
            ha='center',
            va='bottom',
            fontsize=10,
            fontweight='bold',
            color='black'
        )

    # --- Legend (البوكس الجديد) ---
    legend_handles = [
        mpatches.Patch(facecolor=colors[0], hatch=patterns[0], edgecolor='black', label='Control'),
        mpatches.Patch(facecolor=colors[1], hatch=patterns[1], edgecolor='black', label='NDP: Newly Diagnosed Patients'),
        mpatches.Patch(facecolor=colors[2], hatch=patterns[2], edgecolor='black', label='TP: Treated Patients')
    ]

    ax.legend(
        handles=legend_handles,
        loc='upper right',
        fontsize=10,
        frameon=True,
        edgecolor='black',
        framealpha=1,
        fancybox=False
    )

    plt.tight_layout()
    plt.savefig(f'BarChart_{var_name}_Final.png', dpi=300)
    plt.show()

    print(f"✅ تم حفظ الصورة CP بالشكل النهائي!")

# **Box Blot**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np

# 1. قراءة ملف النتائج
try:
    final_df = pd.read_excel('Kruskal_Analysis.xlsx')
    print("✅ تم تحميل ملف نتائج Kruskal بنجاح!")
except FileNotFoundError:
    print("⚠️ ملف 'Kruskal_Analysis.xlsx' مش موجود!")
    final_df = pd.DataFrame()

# التأكد من وجود الداتا الأصلية
if final_df.empty or 'df' not in locals():
    if final_df.empty:
        pass
    else:
        print("⚠️ الداتا الأصلية 'df' مش موجودة في الذاكرة!")
else:
    print(f"🚀 جاري رسم {len(final_df)} Box Plot...")

    groups_labels_on_chart = ['Control', 'NDP', 'TP']
    sorted_groups = sorted(df['Group'].unique())
    result_cols = [c for c in final_df.columns if 'Median' in c and 'Group' in c]

    colors = ['#D3D3D3', '#A9A9A9', '#808080']
    patterns = ['/', '\\', 'x']

    legend_handles = [
        mpatches.Patch(facecolor=colors[0], hatch=patterns[0], edgecolor='black', label='Control'),
        mpatches.Patch(facecolor=colors[1], hatch=patterns[1], edgecolor='black', label='NDP: Newly Diagnosed Patients'),
        mpatches.Patch(facecolor=colors[2], hatch=patterns[2], edgecolor='black', label='TP: Treated Patients')
    ]

    for index, row in final_df.iterrows():
        var_name = row['Variable']
        if var_name not in df.columns:
            continue

        print(f"  - Drawing: {var_name}...")

        data_to_plot = []
        for g_name in sorted_groups:
            vals = df[df['Group'] == g_name][var_name].dropna().values
            data_to_plot.append(vals)

        letters = []
        for col in result_cols:
            val_str = str(row[col])
            try:
                if '(' in val_str:
                    letter = val_str.split('(')[-1].replace(')', '').strip()
                else:
                    letter = ''
                letters.append(letter)
            except:
                letters.append('')

        # --- الرسم ---
        fig, ax = plt.subplots(figsize=(10, 7))

        bplot = ax.boxplot(
            data_to_plot,
            patch_artist=True,
            labels=groups_labels_on_chart,
            widths=0.6,
            flierprops={'marker': 'o', 'markerfacecolor': 'black', 'markersize': 5}
        )

        for patch, color, pattern in zip(bplot['boxes'], colors, patterns):
            patch.set_facecolor(color)
            patch.set_hatch(pattern)
            patch.set_edgecolor('black')

        for element in ['whiskers', 'caps', 'medians']:
            plt.setp(bplot[element], color='black', linewidth=1.2)

        ax.set_xlabel('Group', fontsize=12, fontweight='bold')
        ax.set_ylabel(f'Median {var_name}', fontsize=12, fontweight='bold')
        ax.grid(axis='y', linestyle=':', alpha=0.5, color='gray')
        ax.set_axisbelow(True)

        # --- وضع الحروف في مكان ثابت (فوق الشنب فقط) ---

        # بنحسب مسافة صغيرة ثابتة (Offset) بناءً على طول المحور
        y_min_ax, y_max_ax = ax.get_ylim()
        offset = (y_max_ax - y_min_ax) * 0.03 # مسافة 3% ثابتة فوق الشنب

        for i, letter in enumerate(letters):
            if not letter: continue

            # الكود ده بيجيب مكان "الغطاء" اللي فوق الشنب بالظبط
            # (i * 2) + 1 دي معادلة ثابتة في matplotlib عشان تجيب الشنب العلوي
            top_whisker_y = bplot['caps'][(i * 2) + 1].get_ydata()[0]

            ax.text(
                i + 1,
                top_whisker_y + offset, # المكان: فوق الشنب + المسافة الثابتة
                f"({letter})",
                ha='center',
                va='bottom',
                fontsize=12,
                fontweight='bold',
                color='black'
            )

        ax.legend(
            handles=legend_handles,
            loc='upper right',
            fontsize=10,
            frameon=True,
            edgecolor='black',
            framealpha=1,
            fancybox=False
        )

        plt.tight_layout()
        safe_var_name = "".join([c for c in var_name if c.isalnum() or c in (' ', '-', '_')]).strip()
        plt.savefig(f'BoxPlot_KW_{safe_var_name}.png', dpi=300)
        plt.close()

    print("\n✅ تم الحفظ! الحروف الآن في مكان ثابت وموحد فوق الـ Whiskers.")





# **Roc-Carve**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix

# 1. تحميل الداتا
file_path = 'Data.xlsx'
try:
    df = pd.read_excel(file_path, sheet_name='Clean Data')
except:
    try:
        df = pd.read_excel(file_path)
    except:
        df = pd.DataFrame()

if not df.empty:
    if 'Group' in df.columns:
        df['Group'] = df['Group'].astype(str).str.replace('"', '').str.replace("'", "").str.strip()

    # تحديد المتغيرات
    exclude_cols = ['Group', 'Name', 'Sex', 'id', 'ID']
    target_vars = [col for col in df.select_dtypes(include=[np.number]).columns if col not in exclude_cols]

    # === الإعدادات ===
    comparisons = [
        ('Group 1', 'Group 2', 'Control vs NDP'),
        ('Group 1', 'Group 3', 'Control vs TP'),
        ('Group 3', 'Group 2', 'TP vs NDP')
    ]

    # التمييز (ألوان + أنماط خطوط + أشكال هندسية)
    colors = ['#1f77b4', '#2ca02c', '#d62728']
    linestyles = ['-', '--', '-.']
    markers = ['o', '^', 's']

    print(f"جاري العمل على المتغيرات: {target_vars}")

    for target_var in target_vars:
        try:
            plt.figure(figsize=(10, 8))
            plot_valid = False

            for i, (neg_g, pos_g, label) in enumerate(comparisons):
                sub_df = df[df['Group'].isin([neg_g, pos_g])].dropna(subset=[target_var])

                if sub_df.empty: continue
                plot_valid = True

                y_true = (sub_df['Group'] == pos_g).astype(int)
                y_scores = sub_df[target_var].values

                fpr, tpr, thresholds = roc_curve(y_true, y_scores)
                roc_auc = auc(fpr, tpr)

                if roc_auc < 0.5:
                    y_scores = -y_scores
                    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
                    roc_auc = auc(fpr, tpr)

                J = tpr - fpr
                ix = np.argmax(J)
                best_thresh = thresholds[ix]

                y_pred = (y_scores >= best_thresh).astype(int)
                tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

                sens = tp / (tp + fn) if (tp + fn) > 0 else 0
                spec = tn / (tn + fp) if (tn + fp) > 0 else 0

                label_text = (f"{label} (AUC={roc_auc:.2f})\n"
                              f"Sens:{sens:.1%} Spec:{spec:.1%}")

                # === التعديل الجوهري هنا ===
                # نضع علامة واحدة فقط في منتصف الخط باستخدام markevery مع قائمة بمؤشر واحد
                mid_index = len(fpr) // 2

                plt.plot(fpr, tpr,
                         color=colors[i],
                         linestyle=linestyles[i],
                         marker=markers[i],
                         markevery=[mid_index],      # علامة واحدة فقط في المنتصف
                         markersize=10,              # حجم العلامة كبير وواضح
                         lw=2.5,
                         label=label_text)

                # علامة الـ Cut-off الأفضل (X سوداء)
                plt.scatter(fpr[ix], tpr[ix], marker='X', color='black', s=100, zorder=10)

            if plot_valid:
                plt.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle=':')

                plt.xlim([0.0, 1.0])
                plt.ylim([0.0, 1.05])
                plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12, fontweight='bold')
                plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12, fontweight='bold')

                plt.legend(loc="lower right", fontsize=10, frameon=True, framealpha=0.9)
                plt.grid(alpha=0.3)
                plt.tight_layout()

                safe_name = str(target_var).replace("/", "-").replace("\\", "-")
                filename = f'ROC_{safe_name}.png'
                plt.savefig(filename, dpi=300, bbox_inches='tight')
                plt.close()
                print(f"✅ تم حفظ: {filename}")
            else:
                plt.close()

        except Exception as e:
            print(f"Error processing {target_var}: {e}")
            plt.close()

# **Roc-Carve results**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc, confusion_matrix

# 1. تحميل الداتا
file_path = 'Data.xlsx'
try:
    df = pd.read_excel(file_path, sheet_name='Clean Data')
except:
    try:
        df = pd.read_excel(file_path)
    except:
        df = pd.DataFrame()

if not df.empty:
    print("✅ تم تحميل البيانات بنجاح.")

    # تنظيف اسم الجروب
    if 'Group' in df.columns:
        df['Group'] = df['Group'].astype(str).str.replace('"', '').str.replace("'", "").str.strip()

    # تحديد المتغيرات الرقمية
    exclude_cols = ['Group', 'Name', 'Sex', 'id', 'ID']
    target_vars = [col for col in df.select_dtypes(include=[np.number]).columns if col not in exclude_cols]

    # تعريف المقارنات (نفس المسميات المطلوبة)
    comparisons = [
        ('Group 1', 'Group 2', 'Control vs NDP'),
        ('Group 1', 'Group 3', 'Control vs TP'),
        ('Group 3', 'Group 2', 'TP vs NDP')
    ]

    # قائمة لتخزين النتائج
    results_list = []

    print("⏳ جاري حساب النتائج وتجميعها...")

    for target_var in target_vars:
        for neg_g, pos_g, label in comparisons:
            # فلترة الداتا
            sub_df = df[df['Group'].isin([neg_g, pos_g])].dropna(subset=[target_var])

            if sub_df.empty or sub_df[target_var].nunique() < 2:
                continue

            # تجهيز القيم
            y_true = (sub_df['Group'] == pos_g).astype(int)
            y_scores = sub_df[target_var].values

            # حساب الـ ROC
            fpr, tpr, thresholds = roc_curve(y_true, y_scores)
            roc_auc = auc(fpr, tpr)

            # التحقق من العلاقة العكسية وقلبها إذا لزم الأمر
            is_inverted = False
            if roc_auc < 0.5:
                y_scores = -y_scores
                fpr, tpr, thresholds = roc_curve(y_true, y_scores)
                roc_auc = auc(fpr, tpr)
                is_inverted = True

            # حساب أفضل نقطة (Optimal Threshold)
            J = tpr - fpr
            ix = np.argmax(J)
            best_thresh = thresholds[ix]

            # استرجاع القيمة الأصلية للـ Threshold لو تم قلب الكيرف
            display_thresh = -best_thresh if is_inverted else best_thresh

            # حساب مصفوفة الارتباك (Confusion Matrix)
            y_pred = (y_scores >= best_thresh).astype(int)
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

            # حساب الحساسية والخصوصية
            sens = tp / (tp + fn) if (tp + fn) > 0 else 0
            spec = tn / (tn + fp) if (tn + fp) > 0 else 0
            accuracy = (tp + tn) / (tp + tn + fp + fn)

            # إضافة النتيجة للقائمة
            results_list.append({
                'Variable': target_var,
                'Comparison': label,
                'AUC': round(roc_auc, 3),
                'Cut-off Value': round(display_thresh, 3),
                'Sensitivity (%)': round(sens * 100, 1),
                'Specificity (%)': round(spec * 100, 1),
                'Accuracy (%)': round(accuracy * 100, 1),
                'True Positive (TP)': tp,
                'False Positive (FP)': fp,
                'True Negative (TN)': tn,
                'False Negative (FN)': fn
            })

    # تحويل القائمة إلى DataFrame
    results_df = pd.DataFrame(results_list)

    # حفظ الملف إكسيل
    output_filename = 'ROC_Analysis_Results.xlsx'
    results_df.to_excel(output_filename, index=False)

    print(f"✅ تم حفظ النتائج في ملف: {output_filename}")
    print(results_df.head()) # عرض أول 5 سطور للتأكد

else:
    print("❌ لم يتم تحميل البيانات.")

# **Correlation Analysis HO-1, CP and CEA**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import numpy as np
import itertools

# --- دالة الرسم (مساعدة) ---
def plot_spearman(ax, data, x_var, y_var, title, color):
    valid_data = data[[x_var, y_var]].apply(pd.to_numeric, errors='coerce').dropna()

    if len(valid_data) < 2:
        ax.text(0.5, 0.5, "Not enough data", transform=ax.transAxes, ha='center', va='center')
        # تم إزالة العنوان هنا
        return np.nan, np.nan

    # حساب معامل سبيرمان
    r, p = spearmanr(valid_data[x_var], valid_data[y_var], nan_policy='omit')
    p_text = "p < 0.001" if p < 0.001 else f"p = {p:.3f}"
    stat_text = f"Spearman r = {r:.3f}\n{p_text}"

    # الرسم
    sns.regplot(
        data=valid_data, x=x_var, y=y_var, ax=ax,
        scatter_kws={'alpha': 0.6, 'color': color, 's': 50},
        line_kws={'color': 'black', 'linestyle': '--'}, ci=95
    )

    # إضافة الإحصائيات
    ax.text(0.05, 0.95, stat_text, transform=ax.transAxes, va='top', ha='left',
            fontsize=10, bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7, ec='none'))

    # --- تم إزالة سطر العنوان ---
    # ax.set_title(title, fontsize=12, weight='bold')

    ax.set_xlabel(x_var, fontsize=11)
    ax.set_ylabel(y_var, fontsize=11)
    return r, p

# --- البرنامج الرئيسي ---
try:
    # 1. تحميل البيانات
    file_name = 'Data.xlsx'
    sheet_name = 'Clean Data'

    print(f"Reading file: {file_name}, Sheet: {sheet_name}...")
    # تأكد من مسار الملف الصحيح لديك
    df = pd.read_excel(file_name, sheet_name=sheet_name)

    # 2. تنظيف أسماء الأعمدة
    df.columns = df.columns.str.strip()

    # 3. تنظيف عمود Group
    df['Group'] = df['Group'].astype(str).str.replace('"', '').str.strip()

    group_map = {
        'Group 1': 'Control',
        'Group 2': 'Newly Diagnosed Patients',
        'Group 3': 'Treated Patients'
    }
    df['Group'] = df['Group'].replace(group_map)

    groups = ['Control', 'Newly Diagnosed Patients', 'Treated Patients']
    colors = ['forestgreen', 'goldenrod', 'firebrick']

    # 4. تحديد المتغيرات والأزواج المطلوبة
    variables = ['HO-1', 'CP', 'CEA']
    pairs = list(itertools.combinations(variables, 2)) # ينشئ كل الأزواج الممكنة

    sns.set(style='whitegrid')

    # الدوران على كل زوج من المتغيرات ورسمه
    for x_var, y_var in pairs:
        print(f"Processing: {x_var} vs {y_var}")

        # تحويل البيانات لأرقام
        df[x_var] = pd.to_numeric(df[x_var], errors='coerce')
        df[y_var] = pd.to_numeric(df[y_var], errors='coerce')

        fig, axes = plt.subplots(2, 2, figsize=(14, 12))

        # رسم العينة الكلية (Total Sample)
        plot_spearman(axes[0, 0], df, x_var, y_var, 'Total Sample', 'darkslateblue')

        # رسم المجموعات الفرعية
        for i, group in enumerate(groups):
            row = (i + 1) // 2
            col = (i + 1) % 2
            subset = df[df['Group'] == group]
            plot_spearman(axes[row, col], subset, x_var, y_var, group, colors[i])

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])

        # حفظ الصورة باسم المتغيرين
        file_out = f'spearman_{x_var}_{y_var}.png'.replace(' ', '_')
        plt.savefig(file_out)
        print(f"Saved plot: {file_out}")
        plt.close() # إغلاق الرسم للانتقال للزوج التالي

except Exception as e:
    print(f"\nAn error occurred: {e}")

# **Heatmap (pearson and spearman)**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr, spearmanr

# --- إعدادات الرسم ---
sns.set(style='white')

# --- دالة مساعدة لحساب المصفوفات ---
def calculate_corr_matrix(data, method='pearson'):
    cols = data.columns
    n = len(cols)

    corr_matrix = pd.DataFrame(np.zeros((n, n)), index=cols, columns=cols)
    annot_matrix = pd.DataFrame(np.empty((n, n), dtype=object), index=cols, columns=cols)
    excel_matrix = pd.DataFrame(np.empty((n, n), dtype=object), index=cols, columns=cols)

    for i in range(n):
        for j in range(n):
            c1 = cols[i]
            c2 = cols[j]

            if i == j:
                corr_matrix.iloc[i, j] = 1.0
                annot_matrix.iloc[i, j] = "1.00"
                excel_matrix.iloc[i, j] = "1.00"
                continue

            valid_data = data[[c1, c2]].dropna()

            if len(valid_data) < 2:
                corr_matrix.iloc[i, j] = 0
                annot_matrix.iloc[i, j] = "NaN"
                excel_matrix.iloc[i, j] = "NaN"
                continue

            if method == 'pearson':
                r, p = pearsonr(valid_data[c1], valid_data[c2])
            else:
                r, p = spearmanr(valid_data[c1], valid_data[c2])

            corr_matrix.iloc[i, j] = r

            if p < 0.001:
                p_text = "p<0.001"
            else:
                p_text = f"p={p:.3f}"

            annot_matrix.iloc[i, j] = f"{r:.2f}\n({p_text})"
            excel_matrix.iloc[i, j] = f"{r:.3f} ({p_text})"

    return corr_matrix, annot_matrix, excel_matrix

# --- البرنامج الرئيسي ---
try:
    # تحميل البيانات
    file_name = 'Data.xlsx'
    sheet_name = 'Clean Data'
    df = pd.read_excel(file_name, sheet_name=sheet_name)

    df.columns = df.columns.str.strip()
    numeric_cols = ['HO-1', 'CP', 'CEA', 'Hb', 'Creatinine', 'ALT', 'Age', 'Cu', 'Fe', 'Zn', 'Ca']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df_numeric = df[numeric_cols]

    # حساب المصفوفات
    r_mat_p, annot_mat_p, excel_mat_p = calculate_corr_matrix(df_numeric, method='pearson')
    r_mat_s, annot_mat_s, excel_mat_s = calculate_corr_matrix(df_numeric, method='spearman')

    unified_cmap = 'coolwarm'

    # --- رسم Pearson (بدون عنوان) ---
    plt.figure(figsize=(16, 14))
    sns.heatmap(r_mat_p, annot=annot_mat_p, fmt='', cmap=unified_cmap, center=0, linewidths=.5, cbar_kws={"shrink": .8}, annot_kws={"size": 9})
    plt.tight_layout()
    plt.savefig('heatmap_pearson_no_title.png')
    print("Saved: heatmap_pearson_no_title.png")

    # --- رسم Spearman (بدون عنوان) ---
    plt.figure(figsize=(16, 14))
    sns.heatmap(r_mat_s, annot=annot_mat_s, fmt='', cmap=unified_cmap, center=0, linewidths=.5, cbar_kws={"shrink": .8}, annot_kws={"size": 9})
    plt.tight_layout()
    plt.savefig('heatmap_spearman_no_title.png')
    print("Saved: heatmap_spearman_no_title.png")

    # --- حفظ الإكسيل ---
    output_filename = 'correlation_analysis_results.xlsx'
    with pd.ExcelWriter(output_filename) as writer:
        excel_mat_p.to_excel(writer, sheet_name='Pearson Correlation')
        excel_mat_s.to_excel(writer, sheet_name='Spearman Correlation')
    print(f"Saved Excel: {output_filename}")

except Exception as e:
    print(f"Error: {e}")

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from google.colab import files
import io
import os

print("--- Ultimate Paired T-Test Analysis (Publication Quality) ---")
print("Includes: Mean ± SD, Mean Difference, 95% CI, t-value, P-value")

# 1. تحديد الملف
target_file = 'matched group1&2.xlsx'

if os.path.exists(target_file):
    print(f"✅ الملف '{target_file}' موجود. جاري التحليل...")
else:
    print(f"⚠️ الملف غير موجود. يرجى رفعه الآن:")
    uploaded = files.upload()
    if uploaded:
        target_file = list(uploaded.keys())[0]
    else:
        target_file = None

if target_file:
    try:
        # قراءة الملف
        if target_file.endswith('.csv'):
            df_raw = pd.read_csv(target_file, header=None)
        else:
            df_raw = pd.read_excel(target_file, header=None)

        # تحديد المجموعات
        group_rows = df_raw[df_raw[0].astype(str).str.contains("Group", case=False, na=False)].index.tolist()

        if len(group_rows) >= 2:
            # === استخراج Newly Diagnosed (Before) ===
            idx_g2_header = group_rows[0] + 1
            idx_g2_end = group_rows[1]
            df_before = df_raw.iloc[idx_g2_header+1 : idx_g2_end].copy()
            df_before.columns = df_raw.iloc[idx_g2_header]
            df_before = df_before.dropna(how='all')

            # === استخراج Treated (After) ===
            idx_g3_header = group_rows[1] + 1
            df_after = df_raw.iloc[idx_g3_header+1 : ].copy()
            df_after.columns = df_raw.iloc[idx_g3_header]
            df_after = df_after.dropna(how='all')

            # تصحيح الإزاحة (Shift Fix)
            if 'Name' in df_before.columns:
                 if df_before['Name'].isnull().mean() > 0.5:
                    col_idx_1 = df_before.columns[1]
                    df_before['Name'] = df_before[col_idx_1]

            # تنظيف البيانات
            df_before['Name'] = df_before['Name'].astype(str).str.strip()
            df_after['Name'] = df_after['Name'].astype(str).str.strip()
            df_before.columns = df_before.columns.str.strip()
            df_after.columns = df_after.columns.str.strip()

            # الدمج (Matching)
            merged = pd.merge(df_before, df_after, on='Name', suffixes=('_Before', '_After'))

            # المتغيرات
            exclude = ['Name', 'Sex', 'Group', 'id', 'ID', 'nan']
            potential_cols = [c.replace('_Before', '') for c in merged.columns if '_Before' in c]
            target_vars = [c for c in potential_cols if c not in exclude]

            results = []
            for var in target_vars:
                col_pre = f"{var}_Before"
                col_post = f"{var}_After"

                # تحويل لأرقام
                merged[col_pre] = pd.to_numeric(merged[col_pre], errors='coerce')
                merged[col_post] = pd.to_numeric(merged[col_post], errors='coerce')

                valid = merged[[col_pre, col_post]].dropna()

                if len(valid) < 2: continue

                # 1. Basic Stats
                mean_pre = valid[col_pre].mean()
                sd_pre = valid[col_pre].std()
                mean_post = valid[col_post].mean()
                sd_post = valid[col_post].std()

                # 2. Difference Stats
                diff = valid[col_pre] - valid[col_post]
                mean_diff = diff.mean()
                std_diff = diff.std()
                n = len(valid)
                se_diff = std_diff / np.sqrt(n)

                # 3. T-Test & CI
                if np.allclose(valid[col_pre], valid[col_post]):
                    t_stat, p_val = 0, 1.0
                    ci_lower, ci_upper = 0, 0
                else:
                    t_stat, p_val = stats.ttest_rel(valid[col_pre], valid[col_post])
                    # حساب فترة الثقة 95% للفرق
                    ci = stats.t.interval(0.95, df=n-1, loc=mean_diff, scale=se_diff)
                    ci_lower, ci_upper = ci

                # Significance Stars
                sig = 'NS'
                if p_val < 0.05: sig = '*'
                if p_val < 0.01: sig = '**'
                if p_val < 0.001: sig = '***'

                results.append({
                    'Variable': var,
                    'Newly Diagnosed (Mean ± SD)': f"{mean_pre:.2f} ± {sd_pre:.2f}",
                    'Treated (Mean ± SD)': f"{mean_post:.2f} ± {sd_post:.2f}",
                    'Mean Diff.': f"{mean_diff:.2f}",  # الفرق
                    '95% CI of Diff.': f"({ci_lower:.2f}, {ci_upper:.2f})", # فترة الثقة
                    't-statistic': f"{t_stat:.3f}",
                    'P-value': "< 0.001" if p_val < 0.001 else f"{p_val:.3f}",
                    'Sig.': sig
                })

            if results:
                results_df = pd.DataFrame(results)

                # ترتيب الأعمدة النهائي
                cols = [
                    'Variable',
                    'Newly Diagnosed (Mean ± SD)',
                    'Treated (Mean ± SD)',
                    'Mean Diff.',
                    '95% CI of Diff.',
                    't-statistic',
                    'P-value',
                    'Sig.'
                ]
                results_df = results_df[cols]

                print("\n=== Comprehensive Paired T-Test Report ===")
                display(results_df)

                out_file = 'Paired_Ttest_Full_Report.xlsx'
                results_df.to_excel(out_file, index=False)
                print(f"\n✅ تم حفظ التقرير الشامل: {out_file}")
                files.download(out_file)
            else:
                print("❌ لا توجد بيانات كافية.")

    except Exception as e:
        print(f"❌ حدث خطأ: {e}")

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from google.colab import files
import io
import os

print("--- Ultimate Wilcoxon Signed-Rank Test Analysis (Publication Quality) ---")
print("Includes: Median (IQR), Wilcoxon Statistic, P-value, Significance")

# 1. تحديد الملف المستهدف
target_file = 'matched group1&2.xlsx' # أو .csv

# التحقق من وجود الملف، وإذا لم يوجد نطلب الرفع
if os.path.exists(target_file):
    print(f"✅ الملف '{target_file}' موجود. جاري التحليل...")
else:
    print(f"⚠️ الملف غير موجود. يرجى رفعه الآن:")
    uploaded = files.upload()
    if uploaded:
        target_file = list(uploaded.keys())[0]
    else:
        target_file = None

if target_file:
    try:
        # 2. قراءة الملف
        if target_file.endswith('.csv'):
            df_raw = pd.read_csv(target_file, header=None)
        else:
            df_raw = pd.read_excel(target_file, header=None)

        # 3. تحديد المجموعات
        group_rows = df_raw[df_raw[0].astype(str).str.contains("Group", case=False, na=False)].index.tolist()

        if len(group_rows) >= 2:
            # === استخراج Newly Diagnosed (Before) ===
            idx_g2_header = group_rows[0] + 1
            idx_g2_end = group_rows[1]
            df_before = df_raw.iloc[idx_g2_header+1 : idx_g2_end].copy()
            df_before.columns = df_raw.iloc[idx_g2_header]
            df_before = df_before.dropna(how='all')

            # === استخراج Treated (After) ===
            idx_g3_header = group_rows[1] + 1
            df_after = df_raw.iloc[idx_g3_header+1 : ].copy()
            df_after.columns = df_raw.iloc[idx_g3_header]
            df_after = df_after.dropna(how='all')

            # تصحيح الإزاحة (Shift Fix) للأسماء
            if 'Name' in df_before.columns:
                 if df_before['Name'].isnull().mean() > 0.5:
                    col_idx_1 = df_before.columns[1]
                    df_before['Name'] = df_before[col_idx_1]

            # تنظيف البيانات
            df_before['Name'] = df_before['Name'].astype(str).str.strip()
            df_after['Name'] = df_after['Name'].astype(str).str.strip()
            df_before.columns = df_before.columns.str.strip()
            df_after.columns = df_after.columns.str.strip()

            # الدمج (Matching)
            merged = pd.merge(df_before, df_after, on='Name', suffixes=('_Before', '_After'))

            # تحديد المتغيرات
            exclude = ['Name', 'Sex', 'Group', 'id', 'ID', 'nan']
            potential_cols = [c.replace('_Before', '') for c in merged.columns if '_Before' in c]
            target_vars = [c for c in potential_cols if c not in exclude]

            results = []
            for var in target_vars:
                col_pre = f"{var}_Before"
                col_post = f"{var}_After"

                # تحويل لأرقام
                merged[col_pre] = pd.to_numeric(merged[col_pre], errors='coerce')
                merged[col_post] = pd.to_numeric(merged[col_post], errors='coerce')

                valid = merged[[col_pre, col_post]].dropna()

                if len(valid) < 2: continue

                # 1. حساب Median و IQR (المناسب لاختبار ويلكوكسون)
                def get_median_iqr(series):
                    med = series.median()
                    q1 = series.quantile(0.25)
                    q3 = series.quantile(0.75)
                    iqr = q3 - q1
                    return med, iqr

                med_pre, iqr_pre = get_median_iqr(valid[col_pre])
                med_post, iqr_post = get_median_iqr(valid[col_post])

                # 2. إجراء اختبار Wilcoxon
                # نتحقق أولاً إذا كانت القيم متطابقة تماماً لتجنب الأخطاء
                diff = valid[col_pre] - valid[col_post]
                if (diff == 0).all():
                    stat, p_val = 0, 1.0
                    sig = 'Identical'
                else:
                    try:
                        stat, p_val = stats.wilcoxon(valid[col_pre], valid[col_post])
                        # النجوم للدلالة الإحصائية
                        sig = 'NS'
                        if p_val < 0.05: sig = '*'
                        if p_val < 0.01: sig = '**'
                        if p_val < 0.001: sig = '***'
                    except Exception:
                        stat, p_val = np.nan, np.nan
                        sig = 'Error'

                results.append({
                    'Variable': var,
                    'Newly Diagnosed (Median ± IQR)': f"{med_pre:.2f} ± {iqr_pre:.2f}",
                    'Treated (Median ± IQR)': f"{med_post:.2f} ± {iqr_post:.2f}",
                    'Wilcoxon Stat': f"{stat:.1f}",
                    'P-value': "< 0.001" if p_val < 0.001 else f"{p_val:.5f}",
                    'Sig.': sig
                })

            if results:
                results_df = pd.DataFrame(results)

                # ترتيب الأعمدة
                cols = [
                    'Variable',
                    'Newly Diagnosed (Median ± IQR)',
                    'Treated (Median ± IQR)',
                    'Wilcoxon Stat',
                    'P-value',
                    'Sig.'
                ]
                results_df = results_df[cols]

                print("\n=== Comprehensive Wilcoxon Test Report ===")
                try:
                    from IPython.display import display
                    display(results_df)
                except:
                    print(results_df)

                # حفظ وتنزيل
                out_file = 'Wilcoxon_Test_Full_Report.xlsx'
                results_df.to_excel(out_file, index=False)
                print(f"\n✅ تم حفظ التقرير: {out_file}")
                files.download(out_file)
            else:
                print("❌ لا توجد بيانات كافية.")
        else:
             print("❌ لم يتم العثور على مجموعات (Groups) كافية في الملف.")

    except Exception as e:
        print(f"❌ حدث خطأ أثناء المعالجة: {e}")